In [ ]:
!pip install gradio
!pip install torch
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers
!pip install diffusers
!pip install accelerate
!pip install tiktoken
!pip install -U huggingface_hub
!huggingface-cli login

In [ ]:
import gradio as gr
from diffusers import AutoPipelineForText2Image
import torch
from PIL import Image
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

examples = [
'a chair', 'a cup', 'a watch'
    ]

css = """
body {
    background-color: #A9A9A9;
    background-image: linear-gradient(45deg, #00f, #0f0, #f00, #00f);
    background-size: 400% 400%;
    animation: gradientAnimation 15s linear infinite;
    color: #fff;
    font-family: Arial, sans-serif;
}
/* 标题居中 */
.gr-hostname-container {
    text-align: center;
}

/* 描述居中 */
.gr-description-container {
    text-align: center;
}
/* 标题样式 */
h1 {
    color: #007BFF;
    font-weight: bold;
}

/* 按钮样式 */
.button, button {
    background-color: #007BFF;
    color: white;
    border: none;
    border-radius: 5px;
    padding: 10px 20px;
    text-transform: uppercase;
    letter-spacing: 1px;
    transition: background-color 0.3s ease;
}

.button:hover, button:hover {
    background-color: #0056b3;
    cursor: pointer;
}

/* 输入框样式 */
input[type='text'], textarea {
    border: 2px solid #007BFF;
    border-radius: 5px;
    padding: 10px;
}

/* 输出框样式 */
.output {
    border: 1px #007BFF;
    background-color: white;
    padding: 10px;
    border-radius: 5px;
}

"""


# 指定设备
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cpu":
    print("CUDA is not available. Using CPU instead.")

def generate_image_and_description(prompt):
    try:
        # 图片生成部分
        img_pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to(device)
        img_pipeline.unet.load_attn_procs("TigerHatKth/metal_Lora", weight_name="pytorch_lora_weights.safetensors")
        image = img_pipeline(prompt).images[0]
        del img_pipeline
        print('del img_piepline')

        # 文字描述生成部分
        text_pipeline = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large", device=device)
        image_text = text_pipeline(image)[0]['generated_text']
        del text_pipeline
        print('del text_pipeline')

        # Persona生成部分
        tokenizer = AutoTokenizer.from_pretrained('stabilityai/stablelm-2-zephyr-1_6b', trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained('stabilityai/stablelm-2-zephyr-1_6b', trust_remote_code=True)
        model.to(device)

        prompt_text = "Hi, you are a designer and please help to describe a person in detail that is using the object in a persona format, start with'persona:', include name, age, gender, appearance, profession, interests: " + image_text
        prompt = [{'role': 'user', 'content': prompt_text}]
        inputs = tokenizer.apply_chat_template(
          prompt,
          add_generation_prompt=True,
          return_tensors='pt'
        )
        tokens = model.generate(
          inputs.to(model.device),
          max_new_tokens=512,
          temperature=0.5,
          do_sample=True
        )
        persona_text = str(tokenizer.decode(tokens[0], skip_special_tokens=True))

        del model
        del tokenizer
        print('del model and tokenizer')

        # 查找并提取 "Persona:" 之后的内容
        start_index = persona_text.find("Persona:")
        if start_index != -1:
            persona_info = persona_text[start_index:]
        else:
            persona_info = "Persona section not found in the text."

        # 对文本进行总结生成用户画像摘要
        summarizer = pipeline("summarization", model="Falconsai/text_summarization")
        summary = summarizer(persona_info, max_length=100, min_length=70, do_sample=False)[0]['summary_text']
        del summarizer
        print('del summarizer')

        #生成用户画像照片
        #查找外貌描述部分

        text = persona_info
        start = text.find("Appearance:")
        if start == -1:
          persona_image_prompt = "Appearance section not found."

        #查找下一个类别的起始位置
        categories = ["Profession:", "Interests:", "Name:", "Age:", "Gender:"]
        end = min([text.find(cat, start) for cat in categories if text.find(cat, start) != -1] + [len(text)])
        persona_image_prompt = text[start:end].strip()
        persona_image_prompt =persona_image_prompt.replace("Appearance:", "")



        persona_image_prompt = "photo, a person, " + persona_image_prompt

        persona_img_pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to(device)

        persona_image = persona_img_pipeline(persona_image_prompt).images[0]
        del persona_img_pipeline
        print('del persona_img_pipeline')

        return image, image_text, persona_info, summary, persona_image
    except Exception as e:
        return "Error: " + str(e), "No description available.", "No persona description available.", "No summary available.", None

def main():

    with gr.Interface(
        css=css,
        fn=generate_image_and_description,
        examples=examples,
        inputs=gr.Textbox(label="随便给点提示，生成一个金属风格的物品"),
        outputs=[
            gr.Image(type="pil", label="生成的图片"),
            gr.Textbox(label="生成的文字描述"),
            gr.Textbox(label="生成的用户画像描述"),
            gr.Textbox(label="用户画像摘要"),
            gr.Image(type="pil", label="用户画像图片")
        ],
        title="设计师创意生成器",
        description="输入一个物品提示，生成设计图，用户描述和用户画像，加速创意生成"

    ) as demo:
        demo.launch(debug=True,)

if __name__ == "__main__":
    main()

